バッチサイズに適したJSONLに整形

In [1]:
import json
import path as path

output_file = path.wiki_batched
with open(output_file, 'w', encoding='utf-8') as json_file:
    stack = ""
    with open(path.wiki, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line)
            if len(data["text"]) <= 10: continue
            elif len(data["text"]) <= path.max_seq_len // 2:
                stack += data["text"] + "<pad>"
                if len(stack) > 512:
                    json.dump({"text": stack}, json_file, ensure_ascii=False)
                    json_file.write('\n')
                    stack = ""
            elif len(data["text"]) > path.max_seq_len:
                split_index = [0]
                split_num = len(data["text"]) // path.max_seq_len + 1
                split_size = len(data["text"]) // split_num
                for i in range(split_num-1):
                    split_index.append(data["text"].rfind("。", split_index[i], split_index[i] + split_size)+1)
                    json.dump({"text": data["text"][split_index[i]:split_index[i+1]]}, json_file, ensure_ascii=False)
                    json_file.write('\n')
                json.dump({"text": data["text"][split_index[-1]:]}, json_file, ensure_ascii=False)
                json_file.write('\n')
            else:
                json.dump({"text": data["text"]}, json_file, ensure_ascii=False)
                json_file.write('\n')
            #break

データセットを作成し、PT形式で保存する

In [2]:
import json
import path as path
from charTokenizer import CharTokenizer
from dataset import JPNDataset
import torch
from torch.utils.data import DataLoader

tokenizer = CharTokenizer()
with open(path.charTokenizer, 'r', encoding='utf-8') as f:
    tokenizer.vocab = json.load(f)

dataset = JPNDataset(path.wiki_batched, tokenizer, max_seq_len=path.max_seq_len)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
batch = next(iter(dataloader))
print(batch)

torch.save(dataset, f"../data/dataset_{path.max_seq_len}.pt")

tensor([[ 597, 1728,  484,  ...,    0,    0,    0],
        [2869,   20,  235,  ...,   52,  189,   97],
        [ 132,  330,   38,  ...,    0,    0,    0],
        ...,
        [ 410,  179,   13,  ...,    0,    0,    0],
        [ 781,  262, 1867,  ...,    0,    0,    0],
        [ 543,  731,  543,  ...,    0,    0,    0]])


モデルを学習させ、PTH形式で保存する

In [3]:
from accelerate import Accelerator
from torch.optim import AdamW
from tqdm import tqdm
from bitsandbytes import optim as bnb_optim
import torch
import torch.nn as nn

def train(model, dataloader, vocab_size, epochs=3, lr=3e-4, weight_decay=0.0, early_stop_loss=0.1):
    accelerator = Accelerator()
    device = accelerator.device

    model.to(device)
    optimizer = bnb_optim.AdamW8bit(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss(ignore_index=0)

    model, optimizer, dataloader = accelerator.prepare(model, optimizer, dataloader)
    model.train()
    for epoch in range(epochs):
        pbar = tqdm(dataloader, desc=f"Epoch: {epoch+1}", disable=not accelerator.is_local_main_process)
        for batch in pbar:
            input_ids = batch[:, :-1]
            labels = batch[:, 1:]

            outputs = model(input_ids)

            loss = criterion(outputs.reshape(-1, vocab_size), labels.reshape(-1))


            optimizer.zero_grad()
            accelerator.backward(loss)
            optimizer.step()

            pbar.set_postfix(loss=loss.item())

            if loss.item() < early_stop_loss:
                print(f"Training stopped early at epoch {epoch+1}, batch {pbar.n} due to train_loss < {early_stop_loss}")
                return  # 訓練を中止

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import path as path
from dataset import JPNDataset
from model import Decoder

dataset = torch.load(f"../data/dataset_{path.max_seq_len}.pt", weights_only=False)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

model = Decoder(vocab_size=22217, d_model=512, n_heads=1, n_layers=1, max_seq_len=path.max_seq_len, dropout=0.1)
train(model, dataloader, vocab_size=22217, epochs=3, lr=3e-4, weight_decay=0.01)

torch.save(model.state_dict(), f"../data/model_{path.max_seq_len}.pth")

Epoch: 1:   0%|          | 45/100312 [03:38<131:53:09,  4.74s/it, loss=5.29]

推論

In [ ]:
import torch
import json
import path as path
from charTokenizer import CharTokenizer

model = Decoder(vocab_size=22217, d_model=512, n_heads=1, n_layers=1, max_seq_len=path.max_seq_len)
model.load_state_dict(torch.load(f"../data/model_{path.max_seq_len}.pth"))
model.eval()

tokenizer = CharTokenizer()
with open(path.charTokenizer, 'r', encoding='utf-8') as f:
    tokenizer.vocab = json.load(f)
inputs = tokenizer(
    "荒れた内を避ける為か中間付近までは馬場の中央付近を走行。道中ペースを緩め脚をためると、そのまま直線も先頭でゴールした。逃げての上がりは32.9で上がり最速タイ。 武豊騎手は「ポンと出て、無理に引っ張ることもなく、マイペースで行けた。ラストでひと伸びして能力の高さを感じた」とコメント。ききょうステークス以来実に1年ぶりの勝利を飾り、素質の高さを見せた。続く逆瀬川ステークスでは前走の走りを評価されてか、古馬と同じ55kgの斤量を課された。チャンピオンズカップの裏開催であった為、武豊騎手から吉田隼人騎手に乗り替わり、朝日杯FS以来のコンビ結成となった。スムーズにゲートを出ると、そのまま内3番手を追走。最後直線は力強く抜け出して、2連勝でのOP入りを決めた。吉田隼人騎手は「約1年ぶりに乗せていただきましたが成長しています。出たなりでいい位置をキープできました。抜け出してから、左にもたれる癖はあるが、上がり勝負にも対応してくれました。競馬に幅が広がったし、これからが楽しみです」と振り返った。2023年（4歳）.明け4歳の始動戦に選ばれたのは東京芝2000mのリステッド戦である[白富士ステークスとされた。当日は前走ローズステークス2着と好走したサリエラに次ぐ2番人気に評価された。レースはドーブネが最内枠から好スタートでハナを奪い、武豊のエスコートで1000m59.9秒という絶妙な時計で逃げを打つ。直線に入り粘りの逃げで後続を離すかに思えたが、残り200mあたりから失速。最後は後方から末脚を伸ばしてきたサリエラに交わされた。それでも内を通って伸びてきていたヤマニンサルバムには抜かせず2着を確保した。レース後武豊は「楽なペースだったけどね…。1ハロンぐらい少し距離が長いのかな」とコメントを残し、2000ｍはドーブネにとって距離が長い可能性が示唆された。"
    , max_seq_len=path.max_seq_len, return_tensors="pt")
with torch.no_grad():
    outputs = model(inputs["input_ids"])
    predictions = torch.argmax(outputs, dim=-1)
output_text = tokenizer.decode(predictions[0])
print(output_text)